In [0]:
"""Notebook MVP 02 - Gerar tabela Silver (ciclos_base) a partir do parquet bruto.

Este notebook:
- Le o arquivo bruto de ciclos (parquet) na pasta mvp/dados_bronze.
- Aplica o mesmo tratamento basico do racional antigo:
  - cria coluna Marcacao (timestamp)
  - converte KM para numerico
  - cria colunas Data, Ciclo, Minutos, Inicio_Ideal, Fim_Ideal
- Salva um parquet tratado em mvp/silver/silver_ciclos_base.parquet
"""

import os
from pathlib import Path

import numpy as np
import pandas as pd

# Caminho base do workspace
WORKSPACE_BASE = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp"

# Caminhos usando (para Pandas)
BRONZE_DIR = f"{WORKSPACE_BASE}/dados_bronze"
SILVER_DIR = f"{WORKSPACE_BASE}/silver"

# Caminhos completos dos arquivos
ARQUIVO_BRONZE = f"{BRONZE_DIR}/ciclos_concatenados.parquet"
ARQUIVO_SILVER = f"{SILVER_DIR}/silver_ciclos_base.parquet"

print("=== CONFIGURACAO DE CAMINHOS ===")
print(f"Workspace base: {WORKSPACE_BASE}")
print(f"Diretorio Bronze ): {BRONZE_DIR}")
print(f"Diretorio Silver ): {SILVER_DIR}")
print(f"Arquivo bronze: {ARQUIVO_BRONZE}")
print(f"Arquivo silver de saida: {ARQUIVO_SILVER}")

# Verificar se arquivo bronze existe
if not os.path.exists(ARQUIVO_BRONZE):

    caminhos_alternativos = [
        f"{WORKSPACE_BASE}/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/dados_bronze/ciclos_concatenados.parquet",
        f"/FileStore/shared_uploads/mvp/dados_bronze/ciclos_concatenados.parquet",
        f"{BRONZE_DIR}/ciclos_concatenados.parquet",
    ]
    
    arquivo_encontrado = None
    for caminho_alt in caminhos_alternativos:
        if os.path.exists(caminho_alt):
            arquivo_encontrado = caminho_alt
            print(f"[OK] Arquivo encontrado em caminho alternativo: {caminho_alt}")
            break
    
    if arquivo_encontrado:
        ARQUIVO_BRONZE = arquivo_encontrado
    else:
        raise FileNotFoundError(
            f"Arquivo bronze nao encontrado em: {ARQUIVO_BRONZE}\n"
            "Tentou os seguintes caminhos:\n" + 
            "\n".join([f"  - {c}" for c in caminhos_alternativos])
        )

# Criar diretorio silver se nao existir
os.makedirs(SILVER_DIR, exist_ok=True)
print(f"[OK] Diretorio Silver criado/verificado: {SILVER_DIR}")

def converter_km(km_str):
    """Converte KM em formato texto (ex.: '123+456') para float (123.456).

    Segue a mesma logica utilizada no script analise_ciclos_configuravel.py.
    """
    if pd.isna(km_str) or str(km_str).strip() == "":
        return np.nan
    s = str(km_str)
    if "+" in s:
        partes = s.split("+")
        if len(partes) == 2 and partes[0].strip() != "" and partes[1].strip() != "":
            try:
                return float(partes[0]) + float(partes[1]) / 1000.0
            except ValueError:
                return np.nan
    try:
        return float(s.replace("+", ""))
    except ValueError:
        return np.nan

print("\n=== CARREGANDO DADOS BRONZE ===")
print(f"Lendo parquet bruto de: {ARQUIVO_BRONZE}")
df_raw = pd.read_parquet(ARQUIVO_BRONZE)
print(f"[OK] Linhas carregadas: {len(df_raw):,}")
print(f"Colunas: {list(df_raw.columns)}")

# Renomear colunas para padronizar com o racional antigo
mapeamento_colunas = {
    "DataEv": "DataOcorrencia",
    "HoraEv": "HoraAcionamento",
    "CodRecurso": "CodigoRecurso",
    "Recursos": "Recurso",
    "SiglaRodovia": "Rodovia",
}

df = df_raw.rename(columns=mapeamento_colunas).copy()

# Criar coluna Marcacao (timestamp) a partir de data + hora
print("\n=== PROCESSANDO DADOS ===")
print("Criando coluna Marcacao...")
df["Marcacao"] = pd.to_datetime(
    df["DataOcorrencia"].astype(str) + " " + df["HoraAcionamento"].astype(str),
    errors="coerce",
)

# Converter KM para numerico
print("Convertendo coluna KM para numerico...")
df["KM_original"] = df.get("KM", np.nan)
df["KM"] = df["KM"].apply(converter_km)

# Remover registros sem Marcacao valida
antes = len(df)
df = df[df["Marcacao"].notna()].copy()
depois = len(df)
print(f"[OK] Registros com Marcacao valida: {depois:,} (removidos {antes - depois:,})")

# Colunas derivadas basicas do racional antigo
print("Criando colunas Data, Ciclo, Minutos, Inicio_Ideal, Fim_Ideal...")

df["Data"] = df["Marcacao"].dt.date

df["Ciclo"] = df["Marcacao"].dt.hour // 3 + 1

df["Minutos"] = (
    df["Marcacao"].dt.hour * 60
    + df["Marcacao"].dt.minute
    + df["Marcacao"].dt.second / 60.0
)

df["Inicio_Ideal"] = (df["Ciclo"] - 1) * 180

df["Fim_Ideal"] = df["Ciclo"] * 180

print("\n=== RESUMO DOS DADOS PROCESSADOS ===")
print(f"Periodo: {df['Marcacao'].min()} a {df['Marcacao'].max()}")
print(f"Rodovias: {list(df['Rodovia'].dropna().unique()[:10])}")
print(f"Sentidos: {list(df['Sentido'].dropna().unique()[:10])}")
print(f"Faixa KM: {float(df['KM'].min()):.2f} a {float(df['KM'].max()):.2f}")

colunas_silver = [
    "Marcacao",
    "Data",
    "Rodovia",
    "Sentido",
    "KM",
    "KM_original",
    "Recurso",
    "CodigoRecurso",
    "Ciclo",
    "Minutos",
    "Inicio_Ideal",
    "Fim_Ideal",
]

colunas_existentes = [c for c in colunas_silver if c in df.columns]

df_silver = df[colunas_existentes].copy()

print("\n=== SALVANDO DADOS SILVER ===")
print(f"Salvando silver_ciclos_base.parquet em: {ARQUIVO_SILVER}")
df_silver.to_parquet(ARQUIVO_SILVER, index=False)

# Verificar se arquivo foi criado
if os.path.exists(ARQUIVO_SILVER):
    tamanho_mb = os.path.getsize(ARQUIVO_SILVER) / (1024 * 1024)
    print(f"[OK] Arquivo salvo com sucesso!")
    print(f"  Linhas na tabela Silver: {len(df_silver):,}")
    print(f"  Tamanho do arquivo: {tamanho_mb:.2f} MB")
    print(f"  Colunas: {len(df_silver.columns)}")
else:
    print("[ERRO] Arquivo nao foi criado!")

print("\n=== CONCLUSAO ===")
print("Camada Silver Base criada com sucesso!")
print(f"Arquivo: {ARQUIVO_SILVER}")

=== CONFIGURACAO DE CAMINHOS ===
Workspace base: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp
Diretorio Bronze ): /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze
Diretorio Silver ): /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver
Arquivo bronze: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/ciclos_concatenados.parquet
Arquivo silver de saida: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver/silver_ciclos_base.parquet
[OK] Arquivo encontrado em caminho alternativo: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/dados_bronze/ciclos_concatenados.parquet
[OK] Diretorio Silver criado/verificado: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/silver

=== CARREGANDO DADOS BRONZE ===
Lendo parquet bruto de: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/dados_bronze/ciclos_concate